In [1]:
import pandas as pd
data_path='../data/'
submission_df = (
                        pd.read_csv(data_path+'submission_ambrosm.csv', index_col='id')
                        + pd.read_csv(data_path+'submission_xgboost_kfold.csv',index_col='id')
                        + pd.read_csv(data_path+'submission_lightgbm_kfold.csv',index_col='id')
                )/3
submission_df.to_csv(data_path+'submission_all.csv')